In [23]:
import numpy as np
import pandas as pd
import os

In [24]:
df_path = os.path.join('data', 'Flight-Price.xlsx')

df = pd.read_excel(df_path)
df.head()

,Unnamed: 0,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13108 entries, 0 to 13107
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       13108 non-null  int64  
 1   Airline          13108 non-null  object 
 2   Date_of_Journey  13108 non-null  object 
 3   Source           13108 non-null  object 
 4   Destination      13108 non-null  object 
 5   Route            13107 non-null  object 
 6   Dep_Time         13108 non-null  object 
 7   Arrival_Time     13108 non-null  object 
 8   Duration         13108 non-null  object 
 9   Total_Stops      13107 non-null  object 
 10  Additional_Info  13108 non-null  object 
 11  Price            10463 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usage: 1.2+ MB


In [26]:
df.isna().sum()

Unnamed: 0            0
Airline               0
Date_of_Journey       0
Source                0
Destination           0
Route                 1
Dep_Time              0
Arrival_Time          0
Duration              0
Total_Stops           1
Additional_Info       0
Price              2645
dtype: int64

In [27]:
df = df.dropna()
df[['Date_of_Journey']] = df[['Date_of_Journey']].apply(pd.to_datetime)
df['day'] = df['Date_of_Journey'].dt.day.astype(int)
df['month'] = df['Date_of_Journey'].dt.month.astype(int)
df['Duration_min'] = df['Duration'].str.extract('(\d+)h').fillna(0).astype(int) * 60 + df['Duration'].str.extract('(\d+)m').fillna(0).astype(int)

df = df.drop(columns=['Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration', 'Route', 'Unnamed: 0'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10462 entries, 0 to 10462
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          10462 non-null  object 
 1   Source           10462 non-null  object 
 2   Destination      10462 non-null  object 
 3   Total_Stops      10462 non-null  object 
 4   Additional_Info  10462 non-null  object 
 5   Price            10462 non-null  float64
 6   day              10462 non-null  int64  
 7   month            10462 non-null  int64  
 8   Duration_min     10462 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 817.3+ KB


C:\Users\ARITRA\AppData\Local\Temp\ipykernel_4896\1066056434.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[['Date_of_Journey']] = df[['Date_of_Journey']].apply(pd.to_datetime)


In [28]:
print(df['Airline'].unique())
print(df['Total_Stops'].unique())
print(df['Additional_Info'].unique())
print(df['Source'].unique())
print(df['Destination'].unique())

['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']
['non-stop' '2 stops' '1 stop' '3 stops' '4 stops']
['No info' 'In-flight meal not included' 'No check-in baggage included'
 '1 Short layover' 'No Info' '1 Long layover' 'Change airports'
 'Business class' 'Red-eye flight' '2 Long layover']
['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']
['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad']


In [29]:
stops = {
	'non-stop': 0,
	'1 stop': 1,
	'2 stops': 2,
	'3 stops': 3,
	'4 stops': 4
}

df['Total_Stops'] = df['Total_Stops'].map(stops)

df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,day,month,Duration_min
0,IndiGo,Banglore,New Delhi,0,No info,3897.0,24,3,170
1,Air India,Kolkata,Banglore,2,No info,7662.0,1,5,445
2,Jet Airways,Delhi,Cochin,2,No info,13882.0,9,6,1140
3,IndiGo,Kolkata,Banglore,1,No info,6218.0,12,5,325
4,IndiGo,Banglore,New Delhi,1,No info,13302.0,1,3,285


In [30]:
from sklearn.preprocessing import LabelEncoder

le_airline = LabelEncoder()
le_info = LabelEncoder()
le_airports = LabelEncoder()
airports = ['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai', 'New Delhi', 'Cochin', 'Hyderabad']

df['Airline'] = le_airline.fit_transform(df['Airline'])
df['Additional_Info'] = le_info.fit_transform(df['Additional_Info'])
airport_encoder = le_airports.fit(airports)
df['Source'] = airport_encoder.transform(df['Source'])
df['Destination'] = airport_encoder.transform(df['Destination'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10462 entries, 0 to 10462
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          10462 non-null  int64  
 1   Source           10462 non-null  int64  
 2   Destination      10462 non-null  int64  
 3   Total_Stops      10462 non-null  int64  
 4   Additional_Info  10462 non-null  int64  
 5   Price            10462 non-null  float64
 6   day              10462 non-null  int64  
 7   month            10462 non-null  int64  
 8   Duration_min     10462 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 817.3 KB


In [32]:
df.to_csv('transformed_data.csv', index=False)